In [ ]:
# First cell - Installation and imports
!pip install flask-socketio flask-ngrok pyngrok

# Second cell - Complete application code
from flask import Flask, render_template_string, request, jsonify
from flask_socketio import SocketIO, emit
import sqlite3
from datetime import datetime
from pyngrok import ngrok
from google.colab import output
output.enable_custom_widget_manager()

# Second cell - Get your auth token from https://dashboard.ngrok.com/get-started/your-authtoken
from pyngrok import ngrok
# Replace with your actual token from ngrok dashboard
ngrok.set_auth_token("2rz8bmTBDdJZlIqTeifCts1ehVA_52QbQsmgY9kcgooErYoN")


app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Patient Queue System</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        .priority-Emergency { background-color: #ffcdd2 !important; }
        .priority-Urgent { background-color: #ffe0b2 !important; }
        .priority-Normal { background-color: #fff9c4 !important; }
        .consulted { background-color: #c8e6c9 !important; }
    </style>
</head>
<body>
    <div class="container mt-5">
        <h1 class="text-center">Patient Queue System</h1>
        <form id="patientForm" class="mt-4">
            <div class="mb-3">
                <label for="name" class="form-label">Name</label>
                <input type="text" class="form-control" id="name" required>
            </div>
            <div class="mb-3">
                <label for="age" class="form-label">Age</label>
                <input type="number" class="form-control" id="age" required>
            </div>
            <div class="mb-3">
                <label for="priority" class="form-label">Priority</label>
                <select class="form-control" id="priority" required>
                    <option value="Normal">Normal</option>
                    <option value="Urgent">Urgent</option>
                    <option value="Emergency">Emergency</option>
                </select>
            </div>
            <div class="mb-3">
                <label for="symptoms" class="form-label">Symptoms</label>
                <textarea class="form-control" id="symptoms" rows="3" required></textarea>
            </div>
            <button type="submit" class="btn btn-primary">Add Patient</button>
        </form>

        <h2 class="mt-5">Current Queue</h2>
        <table class="table table-striped mt-3">
            <thead>
                <tr>
                    <th>#</th>
                    <th>Token</th>
                    <th>Name</th>
                    <th>Age</th>
                    <th>Priority</th>
                    <th>Status</th>
                    <th>Action</th>
                </tr>
            </thead>
            <tbody id="queueTable"></tbody>
        </table>
    </div>

    <script src="https://cdn.socket.io/4.5.4/socket.io.min.js"></script>
    <script>
        const socket = io.connect();
        const form = document.getElementById('patientForm');
        const queueTable = document.getElementById('queueTable');

        function updateStatus(patientId) {
            fetch('/update_status', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ id: patientId }),
            }).then(() => fetchQueue());
        }

        function fetchQueue() {
            fetch('/get_queue')
                .then(response => response.json())
                .then(data => {
                    queueTable.innerHTML = '';
                    data.forEach((patient, index) => {
                        const row = document.createElement('tr');
                        row.className = `priority-${patient[3]} ${patient[4] === 'Consulted' ? 'consulted' : ''}`;
                        row.innerHTML = `
                            <td>${index + 1}</td>
                            <td>${patient[0]}</td>
                            <td>${patient[1]}</td>
                            <td>${patient[2]}</td>
                            <td>${patient[3]}</td>
                            <td>${patient[4]}</td>
                            <td>
                                ${patient[4] !== 'Consulted' ?
                                    `<button onclick="updateStatus(${patient[5]})" class="btn btn-success btn-sm">Mark as Consulted</button>` :
                                    '<span class="badge bg-success">Completed</span>'}
                            </td>
                        `;
                        queueTable.appendChild(row);
                    });
                });
        }

        form.addEventListener('submit', event => {
            event.preventDefault();
            const data = {
                name: form.name.value,
                age: form.age.value,
                priority: form.priority.value,
                symptoms: form.symptoms.value,
            };
            fetch('/add_patient', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify(data),
            }).then(() => {
                form.reset();
                fetchQueue();
            });
        });

        socket.on('queue_updated', fetchQueue);
        fetchQueue();
    </script>
</body>
</html>
"""

def init_db():
    conn = sqlite3.connect('hospital.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS patients (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    age INTEGER NOT NULL,
                    priority TEXT NOT NULL,
                    symptoms TEXT NOT NULL,
                    token_number INTEGER NOT NULL,
                    arrival_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    status TEXT DEFAULT 'Pending'
                )''')
    conn.commit()
    conn.close()

def assign_token(priority):
    conn = sqlite3.connect('hospital.db')
    c = conn.cursor()
    c.execute("SELECT MAX(token_number) FROM patients WHERE priority = ?", (priority,))
    last_token = c.fetchone()[0]
    conn.close()
    return (last_token or 0) + 1

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/add_patient', methods=['POST'])
def add_patient():
    data = request.get_json()
    name = data['name']
    age = data['age']
    priority = data['priority']
    symptoms = data['symptoms']
    token = assign_token(priority)

    conn = sqlite3.connect('hospital.db')
    c = conn.cursor()
    c.execute("INSERT INTO patients (name, age, priority, symptoms, token_number) VALUES (?, ?, ?, ?, ?)",
              (name, age, priority, symptoms, token))
    conn.commit()
    conn.close()

    socketio.emit('queue_updated')
    return jsonify({'message': 'Patient added successfully'})

@app.route('/get_queue', methods=['GET'])
def get_queue():
    conn = sqlite3.connect('hospital.db')
    c = conn.cursor()
    c.execute("""SELECT token_number, name, age, priority, status, id
                 FROM patients
                 ORDER BY
                 CASE priority
                    WHEN 'Emergency' THEN 1
                    WHEN 'Urgent' THEN 2
                    ELSE 3 END,
                 arrival_time""")
    queue = c.fetchall()
    conn.close()
    return jsonify(queue)

@app.route('/update_status', methods=['POST'])
def update_status():
    data = request.get_json()
    patient_id = data['id']
    conn = sqlite3.connect('hospital.db')
    c = conn.cursor()
    c.execute("UPDATE patients SET status = 'Consulted' WHERE id = ?", (patient_id,))
    conn.commit()
    conn.close()

    socketio.emit('queue_updated')
    return jsonify({'message': 'Status updated successfully'})

if __name__ == '__main__':
    init_db()
    public_url = ngrok.connect(5000).public_url
    print(f" * ngrok tunnel is running at: {public_url}")
    socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)

# Third cell - Run these commands in a separate cell to stop the tunnel when done
# ngrok.kill()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.7 MB/s eta 0:00:00


 * ngrok tunnel is running at: https://6c48-34-13-146-2.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 09:49:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 09:49:45] "GET /socket.io/?EIO=4&transport=polling&t=PINjwO7 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 09:49:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 09:49:45] "GET /get_queue HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 09:49:46] "POST /socket.io/?EIO=4&transport=polling&t=PINjwY8&sid=Vxf9253QpqUF3ATkAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 09:49:46] "GET /socket.io/?EIO=4&transport=polling&t=PINjwYC&sid=Vxf9253QpqUF3ATkAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 09:49:46] "GET /socket.io/?EIO=4